# Aquisição de dados no portal da transparência da ENTSOE

Como conseguir a API Key para acessar os dados do portal da transparência da ENTSOE:

https://uat-transparency.entsoe.eu/content/static_content/Static%20content/web%20api/how_to_get_security_token.html

## Importando bibliotecas

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import time
from dotenv import load_dotenv
from entsoe import EntsoePandasClient

In [2]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()
API_KEY = os.getenv("ENTSOE_API_KEY", "")

# Configuração do cliente Entsoe
client = EntsoePandasClient(API_KEY)

# Definição de parâmetros
country_code = 'FR'
plot = False
data_limite = '2025-06-30'

## Funções

In [3]:
def preencher_horas_faltantes(df, metodo='spline', order=2):
    """
    Preenche as horas faltantes no índice datetime do DataFrame, interpolando os valores de acordo com o método escolhido.
    Converte o índice para UTC para remover o efeito de horário de verão, e depois retorna ao timezone original.

    Parâmetros:
    df (pd.DataFrame): DataFrame com índice datetime e colunas numéricas.
    metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
                  'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
                  'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
                  'akima', 'cubicspline', 'from_derivatives').
    order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Retorna:
    pd.DataFrame: DataFrame com as horas corrigidas e valores interpolados.
    """

    if df.empty:
        print("O DataFrame está vazio. Nenhuma interpolação foi realizada.")
        return df

    # Copia para não alterar o original
    df = df.copy()

    # Garante que o índice é datetime e remove inválidos
    df.index = pd.to_datetime(df.index, errors='coerce')

    if df.index.isna().any():
        raise ValueError("O índice contém valores não reconhecidos como datas.")

    # Armazena timezone original (se houver)
    tz_original = df.index.tz

    # Converte para UTC se for timezone-aware
    if tz_original is not None:
        df.index = df.index.tz_convert('UTC')
    else:
        df.index = df.index.tz_localize('UTC')  # uniformiza

    # Cria range contínuo de tempo com frequência de 1 hora
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h', tz='UTC')

    # Só reindexa se houver horas ausentes
    if not df.index.equals(full_range):
        df = df.reindex(full_range)

    # Colunas numéricas e não numéricas
    numeric_cols = df.select_dtypes(include=['number']).columns
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns

    if numeric_cols.empty:
        print("Nenhuma coluna numérica encontrada para interpolação. Retornando DataFrame original.")
        return df

    metodos_validos = [
        'linear', 'time', 'index', 'values', 'pad', 'nearest', 'zero',
        'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial', 'krogh',
        'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline', 'from_derivatives'
    ]

    try:
        if metodo in metodos_validos:
            if metodo in ['spline', 'polynomial'] and order is None:
                order = 2
            df[numeric_cols] = df[numeric_cols].interpolate(
                method=metodo,
                order=order if metodo in ['spline', 'polynomial'] else None
            )
        else:
            raise ValueError(f"Método de interpolação inválido: '{metodo}'. Métodos disponíveis: {', '.join(metodos_validos)}")
    except Exception as e:
        print(f"Erro ao interpolar os dados: {e}")
        return df

    if not non_numeric_cols.empty:
        print(f"As colunas não numéricas foram ignoradas: {list(non_numeric_cols)}")

    # Converte de volta para o timezone original
    if tz_original is not None:
        df.index = df.index.tz_convert(tz_original)
    else:
        df.index = df.index.tz_localize(None)

    return df

In [4]:
def get_day_ahead_prices(client, country_code: str, start: pd.Timestamp, end: pd.Timestamp, max_retries: int = 3, metodo='spline', order=2) -> pd.DataFrame:  # type: ignore
    """
    Obtém os preços day-ahead para um país específico em um período determinado, com tentativas automáticas em caso de falha.

    Args:
        client: Cliente de API que fornece os preços.
        country_code (str): Código do país.
        start (pd.Timestamp): Data de início no formato Timestamp.
        end (pd.Timestamp): Data de fim no formato Timestamp.
        max_retries (int): Número máximo de tentativas antes de desistir.
        metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
            'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
            'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
            'akima', 'cubicspline', 'from_derivatives').
            Padrão: 'spline'.
        order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Returns:
        pd.DataFrame: DataFrame contendo os preços day-ahead com timestamps em UTC.
    """

    # Garante que os argumentos start e end são timestamps corretamente formatados
    start = pd.Timestamp(start).tz_localize('UTC') if pd.Timestamp(
        start).tz is None else pd.Timestamp(start)
    end = pd.Timestamp(end).tz_localize('UTC') if pd.Timestamp(
        end).tz is None else pd.Timestamp(end)

    attempts = 0

    while attempts < max_retries:
        try:
            # Obtenção dos dados
            data = client.query_day_ahead_prices(
                country_code, start=start, end=end)

            # Se for uma Series, converte para DataFrame com nome de coluna adequado
            if isinstance(data, pd.Series):
                data = data.to_frame(name='Day Ahead Price')

            # Preenchimento de horas faltantes usando spline de ordem 2
            data = preencher_horas_faltantes(data, metodo=metodo, order=order)

            # Resetar o índice e nomear corretamente a coluna de timestamp
            data = data.reset_index(names=['Timestamp'])

            return data

        except Exception as e:
            attempts += 1
            print(
                f"Erro ao obter os dados de {start.year} (Tentativa {attempts}/{max_retries}): {e}")

            if attempts < max_retries:
                print("Tentando novamente em 5 segundos...")
                time.sleep(5)
            else:
                print(
                    f"Falha após {max_retries} tentativas. Pulando ano {start.year}.")
                # Retorna um DataFrame vazio para evitar falhas no código principal.
                return pd.DataFrame()

In [5]:
def get_load(client, country_code: str, start: pd.Timestamp, end: pd.Timestamp, max_retries: int = 3, metodo='spline', order=2) -> pd.DataFrame:  # type: ignore # type: ignore
    """
    Obtém os a carga atual para um país específico em um período determinado, com tentativas automáticas em caso de falha.

    Args:
        client: Cliente de API que fornece os preços.
        country_code (str): Código do país.
        start (pd.Timestamp): Data de início no formato Timestamp.
        end (pd.Timestamp): Data de fim no formato Timestamp.
        max_retries (int): Número máximo de tentativas antes de desistir.
        metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
            'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
            'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
            'akima', 'cubicspline', 'from_derivatives').
            Padrão: 'spline'.
        order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Returns:
        pd.DataFrame: DataFrame contendo os preços day-ahead com timestamps em UTC.
    """

    # Garante que os argumentos start e end são timestamps corretamente formatados
    start = pd.Timestamp(start).tz_localize('UTC') if pd.Timestamp(
        start).tz is None else pd.Timestamp(start)
    end = pd.Timestamp(end).tz_localize('UTC') if pd.Timestamp(
        end).tz is None else pd.Timestamp(end)

    attempts = 0

    while attempts < max_retries:
        try:
            # Obtenção dos dados
            data = client.query_load(country_code, start=start, end=end)

            # Se for uma Series, converte para DataFrame com nome de coluna adequado
            if isinstance(data, pd.Series):
                data = data.to_frame()

            # Preenchimento de horas faltantes usando spline de ordem 2
            data = preencher_horas_faltantes(data, metodo=metodo, order=order)

            # Resetar o índice e nomear corretamente a coluna de timestamp
            data = data.reset_index(names=['Timestamp'])

            return data

        except Exception as e:
            attempts += 1
            print(
                f"Erro ao obter os dados de {start.year} (Tentativa {attempts}/{max_retries}): {e}")

            if attempts < max_retries:
                print("Tentando novamente em 5 segundos...")
                time.sleep(5)
            else:
                print(
                    f"Falha após {max_retries} tentativas. Pulando ano {start.year}.")
                # Retorna um DataFrame vazio para evitar falhas no código principal.
                return pd.DataFrame()

In [6]:
def get_crossborder_flux(client, country_code: str, start: pd.Timestamp, end: pd.Timestamp, max_retries: int = 3, metodo='spline', order=2) -> pd.DataFrame:  # type: ignore
    """
    Obtém o fluxo de energia de um país específico em um período determinado, com tentativas automáticas em caso de falha.

    Args:
        client: Cliente de API que fornece os preços.
        country_code (str): Código do país.
        start (pd.Timestamp): Data de início no formato Timestamp.
        end (pd.Timestamp): Data de fim no formato Timestamp.
        max_retries (int): Número máximo de tentativas antes de desistir.
        metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
            'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
            'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
            'akima', 'cubicspline', 'from_derivatives').
            Padrão: 'spline'.
        order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Returns:
        pd.DataFrame: DataFrame contendo os preços day-ahead com timestamps em UTC.
    """

    # Garante que os argumentos start e end são timestamps corretamente formatados
    start = pd.Timestamp(start).tz_localize('UTC') if pd.Timestamp(
        start).tz is None else pd.Timestamp(start)
    end = pd.Timestamp(end).tz_localize('UTC') if pd.Timestamp(
        end).tz is None else pd.Timestamp(end)

    attempts = 0

    while attempts < max_retries:
        try:
            # Obtenção dos dados
            export_energy = client.query_physical_crossborder_allborders(
                country_code, start=start, end=end, export=True, per_hour=True).drop(columns='sum')
            import_energy = client.query_physical_crossborder_allborders(
                country_code, start=start, end=end, export=False, per_hour=True).drop(columns='sum')

            # Calcula o fluxo de energia
            data = import_energy.subtract(export_energy, fill_value=0)

            # Se for uma Series, converte para DataFrame com nome de coluna adequado
            if isinstance(data, pd.Series):
                data = data.to_frame()

            # Preenchimento de horas faltantes usando spline de ordem 2
            data = preencher_horas_faltantes(data, metodo=metodo, order=order)

            # Resetar o índice e nomear corretamente a coluna de timestamp
            data = data.reset_index(names=['Timestamp'])

            return data

        except Exception as e:
            attempts += 1
            print(
                f"Erro ao obter os dados de {start.year} (Tentativa {attempts}/{max_retries}): {e}")

            if attempts < max_retries:
                print("Tentando novamente em 5 segundos...")
                time.sleep(5)
            else:
                print(
                    f"Falha após {max_retries} tentativas. Pulando ano {start.year}.")
                # Retorna um DataFrame vazio para evitar falhas no código principal.
                return pd.DataFrame()

In [7]:
def get_generation(client, country_code: str, start: pd.Timestamp, end: pd.Timestamp, psr_type: str = None, max_retries: int = 3, metodo='spline', order=2) -> pd.DataFrame:  # type: ignore
    """
    Obtém os preços day-ahead para um país específico em um período determinado, com tentativas automáticas em caso de falha.

    Args:
        client: Cliente de API que fornece os preços.
        country_code (str): Código do país.
        start (pd.Timestamp): Data de início no formato Timestamp.
        end (pd.Timestamp): Data de fim no formato Timestamp.
        psr_type (str, opcional): Tipo de geração a ser consultado. Se None, retorna todos os tipos disponíveis.
            Os tipos de PSR que podem estar disponíveis são:
                - 'B01' - Biomass
                - 'B02' - Fossil Brown coal/Lignite
                - 'B03' - Fossil Coal-derived gas
                - 'B04' - Fossil Gas
                - 'B05' - Fossil Hard coal
                - 'B06' - Fossil Oil
                - 'B07' - Fossil Oil shale
                - 'B08' - Fossil Peat
                - 'B09' - Geothermal
                - 'B10' - Hydro Pumped Storage
                - 'B11' - Hydro Run-of-river and poundage
                - 'B12' - Hydro Water Reservoir
                - 'B13' - Marine
                - 'B14' - Nuclear
                - 'B15' - Other renewable
                - 'B16' - Solar
                - 'B17' - Waste
                - 'B18' - Wind Offshore
                - 'B19' - Wind Onshore
                - 'B20' - Other
                - 'B21' - AC Link
                - 'B22' - DC Link
                - 'B23' - Substation
                - 'B24' - Transformer
                - 'B25' - Energy storage
        max_retries (int): Número máximo de tentativas antes de desistir.
        metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
            'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
            'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
            'akima', 'cubicspline', 'from_derivatives').
            Padrão: 'spline'.
        order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Returns:
        pd.DataFrame: DataFrame contendo os preços day-ahead com timestamps em UTC.
    """

    # Garante que os argumentos start e end são timestamps corretamente formatados
    start = pd.Timestamp(start).tz_localize('UTC') if pd.Timestamp(
        start).tz is None else pd.Timestamp(start)
    end = pd.Timestamp(end).tz_localize('UTC') if pd.Timestamp(
        end).tz is None else pd.Timestamp(end)

    attempts = 0

    while attempts < max_retries:
        try:
            # Obtenção dos dados
            data = client.query_generation(
                country_code, start=start, end=end, psr_type=psr_type)

            # Renomeando colunas de forma mais intuitiva apenas se houver MultiIndex
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = [
                    f"{source} {metric.replace('Actual ', '')}" for source, metric in data.columns]

            # Tornar negativas as colunas 'Consumption'
            mask = pd.Index(data.columns).str.contains("Consumption", na=False)
            data.loc[:, mask] *= -1

            # Somá-las com 'Aggregated'
            data = (
                data.T.groupby(
                    lambda x: x.replace(" Aggregated", "").replace(
                        " Consumption", "")
                )
                .sum()
                .T
            )

            # Se for uma Series, converte para DataFrame com nome de coluna adequado
            if isinstance(data, pd.Series):
                data = data.to_frame()

            # Preenchimento de horas faltantes usando spline de ordem 2
            data = preencher_horas_faltantes(data, metodo=metodo, order=order)

            # Resetar o índice e nomear corretamente a coluna de timestamp
            data = data.reset_index(names=['Timestamp'])

            return data

        except Exception as e:
            attempts += 1
            print(
                f"Erro ao obter os dados de {start.year} (Tentativa {attempts}/{max_retries}): {e}")

            if attempts < max_retries:
                print("Tentando novamente em 5 segundos...")
                time.sleep(5)
            else:
                print(
                    f"Falha após {max_retries} tentativas. Pulando ano {start.year}.")
                # Retorna um DataFrame vazio para evitar falhas no código principal.
                return pd.DataFrame()

In [8]:
def plot_dataset_plotly(df, title="Gráfico Interativo com Plotly"):
    fig = go.Figure()
    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col))
    fig.update_layout(
        title=title,
        xaxis_title="Tempo",
        yaxis_title="Valores",
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()

In [9]:
def filter_by_date_and_save(
    file_path,
    limit_date=None,
    timezone='Europe/Paris'
):
    """
    Lê o CSV de preços day-ahead com coluna 'Timestamp' contendo timezone,
    converte para o timezone desejado, filtra até a data limite (exclusivo)
    e sobrescreve o arquivo original.

    Parâmetros:
    - file_path (str): Caminho para o arquivo CSV.
    - limit_date (str): Data limite no formato 'YYYY-MM-DD' (exclusivo).
    - timezone (str): Timezone desejado (ex: 'Europe/Paris').
    """
    # Se a data limite não for fornecida a função não faz nada
    if limit_date is None:
        print("Nenhuma data limite fornecida. Nenhum filtro aplicado.")
        return

    # Lê o CSV e converte 'Timestamp' para datetime com UTC
    df = pd.read_csv(file_path, parse_dates=['Timestamp'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True).dt.tz_convert(timezone)

    # Aplica filtro até a data limite no timezone especificado
    limit_date_tz = pd.Timestamp(limit_date, tz=timezone)
    df = df[df['Timestamp'] < limit_date_tz]

    # Sobrescreve o arquivo original
    df.to_csv(file_path, index=False)

## Coleta Day Ahead Prices

In [10]:
# Verifica se o csv já existe
try:
    df = pd.read_csv('./../data/day_ahead_prices.csv')
    print("Arquivo CSV encontrado.")
    if plot:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
        df.set_index('Timestamp', inplace=True)
        plot_dataset_plotly(df)
except FileNotFoundError:
    # Inicializa um dicionário para armazenar os preços por ano
    day_ahead_prices = {}

    # Aquisição dos preços day-ahead para a França de 2015 até o primeiro dado de 2025
    for year in range(2015, 2026):

        start = pd.Timestamp(f'{year}0101', tz='UTC')
        end = pd.Timestamp(f'{year + 1}0101', tz='UTC')

        try:
            prices = get_day_ahead_prices(
                client, country_code=country_code, start=start, end=end)
            day_ahead_prices[year] = prices
            print(f"Dados de {year} adquiridos com sucesso.")
        except Exception as e:
            print(f"Erro ao obter os dados de {year}: {e}")

    # Concatenar todos os DataFrames, tratando duplicatas pela coluna Timestamp
    if day_ahead_prices:
        dahp = pd.concat(day_ahead_prices.values(), ignore_index=True).drop_duplicates(
            subset='Timestamp', keep='last')

        # Armazenar os dados em um arquivo CSV
        dahp.to_csv('./../data/day_ahead_prices.csv', index=False)
        print("Arquivo CSV salvo com sucesso.")
        filter_by_date_and_save(
            file_path='./../data/day_ahead_prices.csv',
            limit_date=data_limite,
            timezone='Europe/Paris')
        if plot:
            dahp['Timestamp'] = pd.to_datetime(dahp['Timestamp'], utc=True)
            dahp.set_index('Timestamp', inplace=True)
            plot_dataset_plotly(dahp)
    else:
        print("Nenhum dado foi adquirido, arquivo CSV não gerado.")

Dados de 2015 adquiridos com sucesso.
Dados de 2016 adquiridos com sucesso.


Connection Error, retrying in 10 seconds


Dados de 2017 adquiridos com sucesso.
Dados de 2018 adquiridos com sucesso.
Dados de 2019 adquiridos com sucesso.
Dados de 2020 adquiridos com sucesso.
Dados de 2021 adquiridos com sucesso.
Dados de 2022 adquiridos com sucesso.
Dados de 2023 adquiridos com sucesso.


Connection Error, retrying in 10 seconds


Dados de 2024 adquiridos com sucesso.
Dados de 2025 adquiridos com sucesso.
Arquivo CSV salvo com sucesso.


## Coleta LOAD

In [11]:
# Verifica se o csv já existe
try:
    df = pd.read_csv('./../data/load.csv')
    print("Arquivo CSV encontrado.")
    if plot:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
        df.set_index('Timestamp', inplace=True)
        plot_dataset_plotly(df)
except FileNotFoundError:
    # Inicializa um dicionário para armazenar os preços por ano
    load = {}

    # Aquisição dos preços day-ahead para a França de 2015 até o primeiro dado de 2025
    for year in range(2015, 2026):

        start = pd.Timestamp(f'{year}0101', tz='UTC')
        end = pd.Timestamp(f'{year + 1}0101', tz='UTC')

        try:
            actual_load = get_load(
                client, country_code=country_code, start=start, end=end, order=3)
            load[year] = actual_load
            print(f"Dados de {year} adquiridos com sucesso.")
        except Exception as e:
            print(f"Erro ao obter os dados de {year}: {e}")

    # Concatenar todos os DataFrames, tratando duplicatas pela coluna Timestamp
    if load:
        all_load = pd.concat(load.values(), ignore_index=True).drop_duplicates(
            subset='Timestamp', keep='last')

        # Armazenar os dados em um arquivo CSV
        all_load.to_csv('./../data/load.csv', index=False)
        print("Arquivo CSV salvo com sucesso.")
        filter_by_date_and_save(
            file_path='./../data/load.csv',
            limit_date=data_limite,
            timezone='Europe/Paris')
        if plot:
            all_load['Timestamp'] = pd.to_datetime(all_load['Timestamp'], utc=True)
            all_load.set_index('Timestamp', inplace=True)
            plot_dataset_plotly(all_load)
    else:
        print("Nenhum dado foi adquirido, arquivo CSV não gerado.")

Dados de 2015 adquiridos com sucesso.
Dados de 2016 adquiridos com sucesso.
Dados de 2017 adquiridos com sucesso.
Dados de 2018 adquiridos com sucesso.
Dados de 2019 adquiridos com sucesso.
Dados de 2020 adquiridos com sucesso.
Dados de 2021 adquiridos com sucesso.
Dados de 2022 adquiridos com sucesso.
Dados de 2023 adquiridos com sucesso.
Dados de 2024 adquiridos com sucesso.
Dados de 2025 adquiridos com sucesso.
Arquivo CSV salvo com sucesso.


## Coleta Actual Generation per Production Type

In [12]:
# Verifica se o csv já existe
try:
    df = pd.read_csv('./../data/gen.csv')
    print("Arquivo CSV encontrado.")
    if plot:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
        df.set_index('Timestamp', inplace=True)
        plot_dataset_plotly(df)
except FileNotFoundError:
    # Inicializa um dicionário para armazenar os preços por ano
    gen = {}

    # Aquisição dos preços day-ahead para a França de 2015 até o primeiro dado de 2025
    for year in range(2015, 2026):

        start = pd.Timestamp(f'{year}0101', tz='UTC')
        end = pd.Timestamp(f'{year + 1}0101', tz='UTC')

        try:
            actual_gen = get_generation(
                client, country_code=country_code, start=start, end=end, order=3)
            gen[year] = actual_gen
            print(f"Dados de {year} adquiridos com sucesso.")
        except Exception as e:
            print(f"Erro ao obter os dados de {year}: {e}")

    # Concatenar todos os DataFrames, tratando duplicatas pela coluna Timestamp
    if gen:
        all_gen = pd.concat(gen.values(), ignore_index=True).drop_duplicates(
            subset='Timestamp', keep='last')

        # Armazenar os dados em um arquivo CSV
        all_gen.to_csv('./../data/gen.csv', index=False)
        print("Arquivo CSV salvo com sucesso.")
        filter_by_date_and_save(
            file_path='./../data/gen.csv',
            limit_date=data_limite,
            timezone='Europe/Paris')
        if plot:
            all_gen['Timestamp'] = pd.to_datetime(all_gen['Timestamp'], utc=True)
            all_gen.set_index('Timestamp', inplace=True)
            plot_dataset_plotly(all_gen)
    else:
        print("Nenhum dado foi adquirido, arquivo CSV não gerado.")

Dados de 2015 adquiridos com sucesso.
Dados de 2016 adquiridos com sucesso.
Dados de 2017 adquiridos com sucesso.
Dados de 2018 adquiridos com sucesso.
Dados de 2019 adquiridos com sucesso.
Dados de 2020 adquiridos com sucesso.
Dados de 2021 adquiridos com sucesso.
Dados de 2022 adquiridos com sucesso.
Dados de 2023 adquiridos com sucesso.
Dados de 2024 adquiridos com sucesso.
Dados de 2025 adquiridos com sucesso.
Arquivo CSV salvo com sucesso.


## Coleta Crossborder Flows

In [13]:
# Verifica se o csv já existe
try:
    df = pd.read_csv('./../data/flux.csv')
    print("Arquivo CSV encontrado.")
    if plot:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
        df.set_index('Timestamp', inplace=True)
        plot_dataset_plotly(df)
except FileNotFoundError:
    # Inicializa um dicionário para armazenar os preços por ano
    flux = {}

    # Aquisição dos preços day-ahead para a França de 2015 até o primeiro dado de 2025
    for year in range(2015, 2026):

        start = pd.Timestamp(f'{year}0101', tz='UTC')
        end = pd.Timestamp(f'{year + 1}0101', tz='UTC')

        try:
            actual_flux = get_crossborder_flux(
                client, country_code=country_code, start=start, end=end, order=3)
            flux[year] = actual_flux
            print(f"Dados de {year} adquiridos com sucesso.")
        except Exception as e:
            print(f"Erro ao obter os dados de {year}: {e}")

    # Concatenar todos os DataFrames, tratando duplicatas pela coluna Timestamp
    if flux:
        all_flux = pd.concat(flux.values(), ignore_index=True).drop_duplicates(
            subset='Timestamp', keep='last')

        # Armazenar os dados em um arquivo CSV
        all_flux.to_csv('./../data/flux.csv', index=False)
        print("Arquivo CSV salvo com sucesso.")
        filter_by_date_and_save(
            file_path='./../data/flux.csv',
            limit_date=data_limite,
            timezone='Europe/Paris')
        if plot:
            all_flux['Timestamp'] = pd.to_datetime(all_flux['Timestamp'], utc=True)
            all_flux.set_index('Timestamp', inplace=True)
            plot_dataset_plotly(all_flux)
    else:
        print("Nenhum dado foi adquirido, arquivo CSV não gerado.")

Dados de 2015 adquiridos com sucesso.


Connection Error, retrying in 10 seconds


Dados de 2016 adquiridos com sucesso.
Dados de 2017 adquiridos com sucesso.
Dados de 2018 adquiridos com sucesso.
Dados de 2019 adquiridos com sucesso.
Dados de 2020 adquiridos com sucesso.
Dados de 2021 adquiridos com sucesso.
Dados de 2022 adquiridos com sucesso.
Dados de 2023 adquiridos com sucesso.
Dados de 2024 adquiridos com sucesso.
Dados de 2025 adquiridos com sucesso.
Arquivo CSV salvo com sucesso.
